In [1]:
from math import sqrt
import pandas as pd
import numpy as np

from pathlib import Path
from collections import defaultdict
import pickle
import gc

from tqdm import tqdm
from pandarallel import pandarallel

from scipy.sparse import csr_matrix, coo_matrix
import implicit
import catboost

import sys
sys.path.append("..")
from src.utils import *
from src.dataset import *
from src.trending import *
from src.level_1 import *

from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from catboost import Pool, CatBoostClassifier, cv
from catboost.utils import get_roc_curve, create_cd
from catboost.eval.catboost_evaluation import CatboostEvaluation

pd.set_option('display.max_colwidth', 255)
tqdm.pandas()
pandarallel.initialize(progress_bar=True, nb_workers=8, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [ ]:
SEED = 1
N = 12
TEST_ON = 1

cv_iteration = 0

dataset = Dataset(skip_days=7 * cv_iteration, test_days=7 * TEST_ON)
train, test = dataset.get_train_and_test()
articles = dataset.get_articles()
customers = dataset.get_customers()

print("Dataset created")

In [4]:
purch_similar_popular = pd.read_csv("../tmp/purch_similar_popular_train.csv", index_col=False)
purch_similar_popular

,customer_id,article_id,purchase_score,similar_parent_purchase_score,als_similarity,general_popular_count,general_popular_count_rank,group_popular_count,group_popular_count_rank
0,0,16003,330,330,212,-1,-1,-1,-1
1,0,16023,28864,28864,98,-1,-1,-1,-1
2,1,42626,0,0,528,-1,-1,-1,-1
3,2,78503,146809,146809,113,-1,-1,-1,-1
4,4,60763,1681,1681,169,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...
56298697,1371979,67543,-1,-1,-1,-1,-1,131,2
56298698,1371979,103304,-1,-1,-1,-1,-1,116,4
56298699,1371979,103033,-1,-1,-1,-1,-1,100,7
56298700,1371979,103303,-1,-1,-1,-1,-1,87,8


In [5]:
test_data = test[["customer_id", "article_id"]]
test_data["target"] = 1

train_data = (
    purch_similar_popular.merge(test_data, on=["customer_id", "article_id"], how="left")
        .fillna({"target": 0.0})
)
print("Train data is done")

<ipython-input-5-824c61a5185a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["target"] = 1


Train data is done


In [7]:
train_data.target.mean(), train_data.target.sum() / test.shape[0]

(0.0007883466421404056, 0.18471480706251484)